# Stock Information

## Overview

## Import Libraries

In [182]:
import pandas as pd
import numpy as np
import yfinance as yf
from urllib.request import urlopen
from pyfinviz import Screener
from pyfinviz import Insider
import pyfinviz
import re

## Stock Analysis

In [170]:
# user defined function to return stock price and volume
def jra_stock_price_volume(ticker, date_0, date_f):
  # get historical data
  df_stock = yf.download(tickers = ticker, start = date_0, end = date_f)
  df_stock = df_stock.reset_index()
  df_stock['Ticker'] = ticker

  return(df_stock)

In [171]:
# set screener options
options = [Screener.CountryOption.USA, Screener.IPODateOption.MORE_THAN_5_YEARS_AGO]

# pages to get
pg_0 = 1
pg_f = 263

pages = [x for x in range(pg_0, pg_f)]

# load finviz screener
screener_obj = Screener(filter_options = options, pages = pages)


In [172]:
# initialize list of screener table results
df_screener_list = []

# build list of screener table results
for i in pages:
  temp1 = screener_obj.data_frames[i]
  df_screener_list.append(temp1)

# join list of screener table results together
df_screener = pd.concat(df_screener_list).reset_index()

df_screener.head()

,index,No,Ticker,Company,Sector,Industry,Country,MarketCap,PE,Price,Change,Volume
0,0,1,A,Agilent Technologies Inc.,Healthcare,Diagnostics & Research,USA,39.51B,32.11,134.84,0.00%,0
1,1,2,AA,Alcoa Corp,Basic Materials,Aluminum,USA,4.89B,-,27.40,0.00%,0
2,2,3,AAAU,Goldman Sachs Physical Gold ETF,Financial,Exchange Traded Fund,USA,-,-,19.92,0.00%,0
3,3,4,AADI,Aadi Bioscience Inc,Healthcare,Biotechnology,USA,46.61M,-,1.90,0.00%,0
4,4,5,AADR,AdvisorShares Dorsey Wright ADR ETF,Financial,Exchange Traded Fund,USA,-,-,57.19,0.00%,0


In [173]:
# list all screened stocks
stock_list = df_screener['Ticker']
stock_list

0          A
1         AA
2       AAAU
3       AADI
4       AADR
        ... 
5231     ZUO
5232    ZVRA
5233     ZWS
5234    ZYME
5235    ZYXI
Name: Ticker, Length: 5236, dtype: object

In [175]:
# # initialize start date
# date_0 = '2018-01-01'

# # initialize end date
# date_f = '2023-02-15'

# # stock code/ticker
# ticker = stock_list[5237]

# jra_stock_price_volume(ticker, date_0, date_f)

# # get historical stock price and volume for screend stock
# df_stock = jra_stock_price_volume(ticker, date_0, date_f)
# df_stock

### Get Insider information

In [176]:
# filter options
options = Insider.FilterOption.BUY
# load finviz insider object
insider_obj = Insider(filter_option = options)

In [177]:
df_insider = insider_obj.table_df
df_insider

,Ticker,Owner,Relationship,Date,Transaction,Cost,Shares,Value,SharesTotal,SECForm4
0,BBGI,BEASLEY BRIAN E,Chief Operating Officer,Feb 15,Buy,0.85,"1,200","1,020","337,636",Feb 20 09:16 AM
1,ORI,ADACHI BARBARA,Director,Feb 16,Buy,28.60,"1,925","55,055","7,845",Feb 20 08:39 AM
2,RZLT,Evans Daron,CFO,Feb 16,Buy,1.37,"9,000","12,330","4,500",Feb 20 07:01 AM
3,RZLT,Evans Daron,CFO,Feb 16,Buy,1.30,"41,900","54,411","70,900",Feb 20 07:01 AM
4,ADC,RAKOLTA JOHN JR,Director,Feb 16,Buy,57.00,"20,430","1,164,510","392,549",Feb 20 06:30 AM
...,...,...,...,...,...,...,...,...,...,...
195,PKBK,Hill Jonathan D.,"SVP, Chief Financial Officer",Feb 13,Buy,16.00,30,480,30,Feb 14 01:32 PM
196,EBF,BURNETT VERA,CFO and Treasurer,Feb 14,Buy,20.47,716,"14,657","4,835",Feb 14 12:55 PM
197,MBCN,Moeller Jennifer L,Director,Feb 14,Buy,23.75,280,"6,650","1,664",Feb 14 12:51 PM
198,CBU,Karaivanov Dimitar,President & CEO,Feb 14,Buy,43.70,"1,000","43,700","16,265",Feb 14 12:12 PM


In [180]:
# get insider stock information
df_insider = insider_obj.table_df
df_insider.Shares = [int(re.sub(pattern = ',', repl = '', string = str(x))) for x in df_insider.Shares]
df_insider.Value = [int(re.sub(pattern = ',', repl = '', string = str(x))) for x in df_insider.Value]
df_insider.Cost = [float(str(x)) for x in df_insider.Cost]
df_insider.SharesTotal = [int(re.sub(pattern = ',', repl = '', string = str(x))) for x in df_insider.SharesTotal]

# aggregate insider stock info
df_insider = (
  df_insider
  .groupby(['Ticker'])[['Value', 'Cost']]
  .agg(["sum", "min", "mean", "max"])
)

# convert column nmaes to flat index 
df_insider.columns =  ["_".join(x) for x in df_insider.columns.to_flat_index()]
df_insider = df_insider.reset_index()

# sort values by value in descending
df_insider = df_insider.sort_values(by = "Value_sum", ascending = False)

# supress scientific notation
pd.options.display.float_format = '{:.0f}'.format

In [181]:
df_insider[0:20]

,Ticker,Value_sum,Value_min,Value_mean,Value_max,Cost_sum,Cost_min,Cost_mean,Cost_max
68,SOC,96200055,99705,13742865,60000000,73,10,10,12
43,LRMR,37521993,22000,18760996,37499993,20,9,10,11
55,PLCE,35528552,10779027,17764276,24749525,27,11,13,16
73,TNYA,19999998,9999999,9999999,9999999,9,4,4,4
41,KYTX,15468992,5568992,7734496,9900000,44,22,22,22
46,MGX,13025000,1025000,6512500,12000000,25,10,13,15
3,AREN,11999999,11999999,11999999,11999999,2,2,2,2
47,MLYS,7499992,7499992,7499992,7499992,14,14,14,14
13,BIGZ,6430748,2504871,3215374,3925877,16,8,8,8
25,ECAT,4765682,585058,1588561,3095857,50,17,17,17
